In [1]:
#imports(if needed)

Main Question: Which Games did the Cougar Men's Basketball Team Perform the best and which games did they perform the worst based on the point difference (points scored - points allowed to score)? 
For this project we will compare using a Binary Search tree and a heap  to find a specific game as well as the max and min score difference. I also want to sort and return all of the games based on the score difference.

The Data:
 Data comes from a .csv file sourced from https://www.sports-reference.com/cbb/schools/washington-state/men/2024-gamelogs-advanced.html. I loaded it into Microsoft Excel and cleaned it dropping all of the stats collected for each game except the scores, date, school played, and location of the game. There were no missing or null values as the season has now ended and all of the data from each game was present. A few of the column names were not named in a way that would allow for easy access so I had to rename the WorL column as well as the dates. Additiionally, the location of the games is denoted using symbols. Previously it was left blank for a home game, had an @ symbol for an away game and an n for neutral venues. I decided to change the symbols to capital letters and add H for home games instead of having a blank value.

 Below is the code for our nodes that will hold the data for each game. The attributes stored are:
 game_number: The id for the game assigned cumulativly (1 is the first game played...)
 date_played: The date the game was played
 location: The location where the game was played
 opponent: Which team we played
 WorL: Whether the game was a Win or a Loss
 team_score: The score of the cougars at the end of the game
 opponent_score: The score of the opposing team at the end of the game
 score_difference: The difference in points scoreed betweeen the cougars and the opposing team calculated by team_score - opponent_score
 The other attributes are used for the data structures.

In [2]:
class Node: #this the node structure that will hold our individual data points from each game. We will read games into nodes from the csv file later.
    def __init__(self, game_number, date_played, location, opponent, WorL, team_score, opponent_score):
        self.game_number = game_number
        self.date_played = date_played
        self.location = location
        self.opponent = opponent
        self.WorL = WorL
        self.team_score = team_score
        self.opponent_score = opponent_score
        self.score_difference = team_score - opponent_score
        #below are helper attributes for the data structures used to store the nodes.
        self.left = None
        self.right = None
        
    def __str__(self):
        return f'{self.game_number}, {self.date_played}, {self.location}, {self.opponent}, {self.WorL}, {self.team_score}, {self.opponent_score}, {self.score_difference}'

Binary Search Tree (BST):
Below is the code for the binary search tree. Essentially we start with a data point as our "root" or starting point. Then when inserting data we insert points "under" the node as a "child" to the left or right based on whether a specified value is greater than or less than the roots specified value. Because a node can only have two children if a node has 2 children already we just move "down" a node and insert there instead. This means that a binary search tree will gain height as we add nodes. 

Important notes, because of the left and right rules the maximum and minimum values will always be the left and rightmost nodes of the tree meaning the time spent to find the minimum and maximum values is directly related to the vertical height of our tree.

Searching for nodes is different however because we have to individually check the value of each node assisted by the fact that we can figure out where to search more efficenetly based on whether the value of the current examined node is greater than or less than our searched for value if that is the information we construct our tree around, otherwise it is random as wee search the tree whether we find it or not.

In [3]:
#Binary Search Tree 
class BST:
    def __init__(self):
        self.root = None
    
    def insert(self, node):
        "This function inserts a node using the _insert helper function."
        if self.root is None: #if there is no root then create one.
            self.root = node
        else: #if there is a root use the helper function to insert the node to the right place.
            self._insert(self.root, node)
            
    def _insert(self, current_node, new_node):
        """This helper function takes a current node and a node to be placed. It then checks the values of the nodes to decide whether the new node
        should be placed to the left or right of the current node based on whether it is smaller or larger than the current node."""
        if new_node.score_difference < current_node.score_difference:
            if current_node.left is None: #does the node have a child to the left?
                current_node.left = new_node #no: Insert to the left
            else:
                self._insert(current_node.left, new_node) #yes: then check again with the left child as the current node.
        else:
            if current_node.right is None: #does the node have a child to the right?
                current_node.right = new_node #no
            else:
                self._insert(current_node.right, new_node) #yes
    
    def get_min(self):
        """This function traverses the tree to find the left most node which is the minimum value
        @return the minimum value node."""
        current_node = self.root
        while current_node.left != None:
            current_node = current_node.left
        return current_node
    
    def get_max(self):
        """This function traverses the tree to find the right most node which is the maximum value
        @return the maximum value node"""
        current_node = self.root
        while current_node.right != None:
            current_node = current_node.right
        return current_node
    
    def height(self):
        """This function returns the height of a the BST using a th _height helper function.
        @return the height of the BST."""
        return self._height(self.root) #call helper function with the root node as the starting node.

    def _height(self, starting_node):
        """This function takes a starting and follows the left and right child nodes down to find the distance from the starting node to the furthest node.
        @return the distance between the starting node and the furthest child node."""
        if starting_node is None: #no starting node
            return -1
        left_height = self._height(starting_node.left) #check height going to left
        right_height = self._height(starting_node.right) #check height going to right
        return 1 + max(left_height, right_height) #return which ever heeght is greater, left or right + 1 to account for the currrent node.
            
    def search(self, value):
        """This function searches for a node with a specific value in the BST using the _search helper function.
        @param, the value to be searched for in the BST.
        @return the node with the value if it exists, otherwise None."""
        return self._search(self.root, value)

    def _search(self, node, value):
        if node is None:
            return None  #Value not found
        elif value < node.score_difference:
            return self._search(node.left, value)  #Value might be in the left subtree
        elif value > node.score_difference:
            return self._search(node.right, value)  #Value might be in the right subtree
        else:
            return node  #Value found
        
    def in_order_traversal(self):
        """This function performs an in order traveral of the BST sing the _in_order_traversal helper function. This will sort the data in order and return it."""
        return self._in_order_traversal(self.root, [])
    
    def _in_order_traversal(self, node, sorted_data):
        """This function performs an in order traversal of the BST and appends the data to a list."""
        if node is not None:
            self._in_order_traversal(node.left, sorted_data)
            sorted_data.append(node)
            self._in_order_traversal(node.right, sorted_data)
        return sorted_data
            
        


Heap:
The code below defines the heap data structure, specifically the Max Heap structure. Like the binary search tree, it is based on a tree like structure with a root or top node that has child nodes under it. In this case, instead of choosing an arbitrary node as our starting value we have the largest value as our root (In a Min Tree we would choose the smallest value.) Afterwards, the nodes are placed under making sure that child nodes are always lesser than or equal to the value of the parent nodes. 

This structure leads to more uncertain traversal of the tree, as there are no left right rules for child nodes meaning the data is assigned in whichever order its read in. The maximum or minimum value is easy to acceess depending on the type (min or max) of the heap because it will be the root node, however only one will be easily accessable while the other will be one of the nodes on the bottom most layer of the tree.

Compared to the BST, insertion and removal of data is significantly faster as the trade off for losing access to easier min and max finding.


In [4]:
#Code for Heap
class Heap:
    def __init__(self):
        self.heap = []
        
    def build_heap(self, data):
        """This function takes a set of nodes and build it into a heap using the heapify function.
        @param data, the data to be built into a heap."""
        self.heap = data[:]
        for i in range(len(self.heap)// 2, -1, -1):
            self.heapify(i)
            
    def heapify(self, i):
        left = 2 * i + 1
        right = 2 * i + 2
        largest = i
        if left < len(self.heap) and self.heap[left].score_difference > self.heap[largest].score_difference:
            largest = left
        if right < len(self.heap) and self.heap[right].score_difference > self.heap[largest].score_difference:
            largest = right
        if largest != i:
            self.heap[i], self.heap[largest] = self.heap[largest], self.heap[i]
            self.heapify(largest)
            
    def search(self, value):
        """This function searches for a node with a specific value in the heap, it will return the node if found or None if not found.
        @param value, the value to bee search for in the heap
        @return the nodee with the value or None."""
        for count in range(len(self.heap)):
            if self.heap[count].score_difference == value:
                return self.heap[count]
        return None
        
    def get_max(self):
        """This function return the root node of the heap which is the maximum value."""
        return self.heap[0]
    
    def heap_sort(self):
        """This function sorts the heap using the heapify function.
        @return a list of the sorted heap."""
        size = len(self.heap)
        sorted_heap = []
        for _ in range(size):
            self.heap[0], self.heap[size - 1] = self.heap[size - 1], self.heap[0]
            sorted_heap.append(self.heap.pop())
            size -= 1
            self.heapify(0)
        return sorted_heap
       
            
        

Now that we have our structures defined, lets load our data from the .csv into the structures and perform our analysis by searching for the min and max score values as well a a game based on a random trait.

In [5]:
#load data from csv file into nodes and create our Binary Search Tree.
with open("MensBasketBallStats.csv", "r") as file:
    node_list = []
    next(file) #skip the header line
    for line in file: #Go line by line through the file and create a node for each line.
        data = line.split(",")
        node = Node(int(data[0]), data[1], data[2], data[3], data[4], int(data[5]), int(data[6]))
        node_list.append(node) #add nodes to the node list.

#Create a BST and insert the nodes
bst = BST()
for node in node_list:
    bst.insert(node)
    
#Create a heap and build it with the nodes
heap = Heap()
heap.build_heap(node_list)


We now have a BST and a Heap built and populated with the data from our nodes. Lets extract our min and max values, search for some games, and print the games in order of score difference using our different data structures.

In [6]:
#Operations of BST

#find and print max and min
minimum_node = bst.get_min()
maximum_node = bst.get_max()
print(f"Highest Score Difference Game: {maximum_node}")
print(f"Lowest Score Difference Game: {minimum_node}")

#search for score differences
search1 = bst.search(0)
search2 = bst.search(8)
search3 = bst.search(17)
print(f"Searched Games:\n10 point difference: {search1}\n8 point difference: {search2}\n17 point difference: {search3}")

#sort list
sorted_data = bst.in_order_traversal()
print("Sorted Data:")
for node in sorted_data:
    print(node)



Highest Score Difference Game: 5, 11/24/23, H, Utah Tech, W, 93, 53, 40
Lowest Score Difference Game: 12, 12/29/23, @, Utah, L, 58, 80, -22
Searched Games:
10 point difference: None
8 point difference: 16, 1/10/24, @, USC, W, 72, 64, 8
17 point difference: 32, 3/14/24, N, Stanford, W, 79, 62, 17
Sorted Data:
12, 12/29/23, @, Utah, L, 58, 80, -22
3, 11/18/23, N, Mississippi State, L, 64, 76, -12
28, 2/24/24, @, Arizona State, L, 61, 73, -12
35, 3/23/24, N, Iowa State, L, 56, 67, -11
10, 12/16/23, N, Santa Clara, L, 61, 69, -8
13, 12/31/23, @, Colorado, L, 67, 74, -7
19, 1/20/24, @, California, L\'a0(1 OT), 75, 81, -6
31, 3/7/24, H, Washington, L, 68, 74, -6
33, 3/15/24, N, Colorado, L, 52, 58, -6
15, 1/6/24, H, Oregon, L, 84, 89, -5
17, 1/13/24, H, Arizona, W, 73, 70, 3
22, 2/3/24, @, Washington, W\'a0(1 OT), 90, 87, 3
27, 2/22/24, @, Arizona, W, 77, 74, 3
29, 2/29/24, H, USC, W, 75, 72, 3
11, 12/21/23, N, Boise State, W, 66, 61, 5
34, 3/21/24, N, Drake, W, 66, 61, 5
23, 2/8/24, @, Oreg

In [7]:
#Operations of Heap

#Find and print max and min
maximum_node = heap.get_max()
print(f"Highest Score Difference Game: {maximum_node}")

#search for score differences
search1 = heap.search(0)
search2 = heap.search(8)
search3 = heap.search(17)
print(f"Searched Games:\n10 point difference: {search1}\n8 point difference: {search2}\n17 point difference: {search3}")

#sort list.
sorted_data = heap.heap_sort()
print("Sorted Data:")
for node in sorted_data:
    print(node)

Highest Score Difference Game: 5, 11/24/23, H, Utah Tech, W, 93, 53, 40
Searched Games:
10 point difference: None
8 point difference: 16, 1/10/24, @, USC, W, 72, 64, 8
17 point difference: 32, 3/14/24, N, Stanford, W, 79, 62, 17
Sorted Data:
5, 11/24/23, H, Utah Tech, W, 93, 53, 40
8, 12/6/23, H, UC-Riverside, W, 86, 49, 37
1, 11/6/23, H, Idaho, W, 84, 59, 25
20, 1/24/24, H, Utah, W, 79, 57, 22
4, 11/19/23, N, Rhode Island, W, 78, 57, 21
25, 2/15/24, H, California, W, 84, 65, 19
9, 12/10/23, H, Grambling, W, 83, 65, 18
2, 11/10/23, H, Prairie View, W, 83, 65, 18
32, 3/14/24, N, Stanford, W, 79, 62, 17
18, 1/18/24, @, Stanford, W, 89, 75, 14
26, 2/17/24, H, Stanford, W, 72, 59, 13
30, 3/2/24, H, UCLA, W, 77, 65, 12
6, 11/27/23, H, Eastern Washington, W, 82, 72, 10
7, 12/2/23, H, Portland State, W, 71, 61, 10
21, 1/27/24, H, Colorado, W, 78, 69, 9
16, 1/10/24, @, USC, W, 72, 64, 8
14, 1/4/24, H, Oregon State, W, 65, 58, 7
23, 2/8/24, @, Oregon State, W, 64, 58, 6
24, 2/10/24, @, Oregon, 

Operation Review:

Comparing finding the maximum and minimum value games we encounter our first limitation. Using the Binary Search Tree, it is a pretty quick operation with a big o notation of O(h) where h is the height of the tree meaning that because we just follow the tree down to the left or right the time we take is dependant on the number of nodes we traverse vertically. Using the Heap, it is much quicker with a big o notation of O(1) because the max or min is always the top most node. However the limitation is that depending on the type of heap you can only find the max or min in O(1) while the other is much slower because you have to search the entire tree for the other value.

When searching values, there are more differences, with the heap, the time complexity is directly related to the number of elements in the heap, O(n) where n is the number of elements. The BST however is slightly more complicated. Because my BST code doesn't ensure balance (the height of left and right subtrees only differ by 1 when inserting values) my tree can end skewed. This changes the time complxity from O(log(n)) for a balanced tree to O(n) which matches the speed of the heap. So for the searchng of values both structures perform exactly the same athough the BST could be optimized later to make it faster for larger values of n. Changing my BST to a balanced tree likely wouldn't garner massive results right away because I only have 36 data points, but if expanded to include more points, the O(log(n)) vs O(n) makes a huge difference.

Finally, the sorting of the lists. For creating a sorted list of the BST elements, we simply just traverse the tree in order since the values are placed to the left and right of the nodes based on whether they are greater than or less than the element above. Because of this property we simply traverse the nodes in a specific order, getting the left child(the lesser value) the root(the middle value) and the right(the greater value) this process is repeated for all nodes in the tree and will visit them in ascending order. Once again for the BST this leads to a time complexity controled by the number of nodes in each tree. For the heap we have access to the interesting Heap Sort method. Because the maximum value (or minimum if we had used a min heap) is the root node we can simply "pop" that off of the stack and re-heapify our list. This gives a time complexity of O(n log n) where n is the number of nodes. Compared to the BST time complexity this is worse for sets that have more than 10 items.

So you may ask, if the BST beats the time complexity of the heap in all of these operations then what is the point of using a heap anyway?

Heaps vastly outperform the BST when it comes to adding and removing values. When a value is added to the BST it take O(n) where n is the number of nodes because every time you add a value you have to traverse the tree to find exactly where it belongs. With the Heap because its looser with its placement rules, you can add a value to the bottom of the heaps subtrees and simply check if it violates the heaps rules. If the value is bigger (or smaller if its a min heap) you simply just move it up a row. This means that in a worst case scenario you will end up moving a value from the bottom of the tree to the top creating a time complexity of O(log(n)) as the number of elements in each row doubles as you add rows. This means that when comparing big and unbalanced BST trees to Heaps the heap has a remarkable advantage when removing and adding elements.

For this project however that doesn't really matter. It is safe to conclude that because I am working with a small data set that doesn't change or have the requirement of adding or removing values the Binary Search Tree is the best structure when compared to the Heap. If the project were interested in updating the values or adding and removing data as it became relevant or irrelevant or processing important values in an order then the heap would be the right pick.